In [11]:
import math
import numpy as np

DEBUG = 0 # 1 = print debug information, 2=detail steps information 
INPUT_FILE = 'hmm-data.txt'
DECIMAL_PRECISION = 1 #decimal place is 0.1**1
GRID_WORLD_ROW = 10 # Y axis row
GRID_WORLD_COLUMN = 10 # X axis elements in each row
GRID_WORLD = []
TOWER_NUMBERS = 4
TOWER_LOCATION = []
NOISY_DISTANCES = []
TIME_STEPS = 11
NOISY_MIN = 0.7
NOISY_MAX = 1.3

STATES = []
OBSERVATIONS = []
INITIAL_PROB = []
TRANSITION_MATRIX = []
DISTANCE_MATRIX = []
EVIDENCE_MATRIX = []

STATE_SEQUENCE=[]

VALIDCELL2COORDINATION={}


def load_parameters(file_name):
    _gw = []
    _tl = []
    _nd = []
    with open(file_name) as f:
        f.readline()   # skip Grid-World 标题行
        f.readline()   # skip the blank line 标题下的空白行
        #得到 方格图
        for _ in range (GRID_WORLD_ROW):
            _gw.append([int(_x) for _x in f.readline().split()])
            
        f.readline()   # skip the blank line
        f.readline()   # skip the blank line
        f.readline()   # skip Tower Locations:
        f.readline()   # skip the blank line
        #得到 塔 坐标
        for _ in range (TOWER_NUMBERS):
            _tl.append([int(_x) for _x in f.readline().split()[2:]]) #skip two label words.
                     
        f.readline()   # skip the blank line
        f.readline()   # skip the blank line
        f.readline()   # skip Noisy Distances to towers 1, 2, 3, 4:
        f.readline()   # skip the blank line
        #得到 噪声
        for _ in range (TIME_STEPS):
            _nd.append([float(_x) for _x in f.readline().split()])
                                            
        return _gw, _tl, _nd

def Print_List(l):
    for i in l:
        print(i)

def NextStepProb(x, y, grid_world):
    #已知是相邻的两个cell，在 cell 1 情况下，走到 cell 2 的概率
    _a = 0#统计cell 1可以走的方向，1/方向数=在 cell 1 情况下走到 cell 2 的概率 

    if((y-1 >=0) and (grid_world[x][y-1]==1)):
        # Left cell is available.
        _a += 1
    else: pass    
    
    if ((y+1 < GRID_WORLD_COLUMN) and (grid_world[x][y+1]==1)):
        # Right cell is available.
        _a += 1
    else: pass 
            
    if ((x-1 >=0) and (grid_world[x-1][y]==1)):
        # Up cell is available.
        _a += 1
    else: pass

    if ((x+1 < GRID_WORLD_ROW) and (grid_world[x+1][y]==1)):
        # Down cell is available.
        _a += 1
    else: pass 
            
    if (_a > 0):
        return 1/_a    
    else:
        return 0  
      
    
def GetProbability(x_1, y_1, x, y, grid_world):#在 cell 1 情况下，走到 cell 2 的概率 
    _a = 0 # available free cells based on given position x_1, y_1
    _n_x = 0 #neighbor in x axis
    _n_y = 0 #neighbor in y axis
  
    if((x-1 == x_1) and (y == y_1)):#向右走一步
        # Are (x, y) next to (under) the (x_1, y_1)?
        return NextStepProb(x_1, y_1, grid_world)
    elif ((x+1 == x_1) and (y == y_1)):#向左走一步
        # Are (x, y) next to (at top of) the (x_1, y_1)?
        return NextStepProb(x_1, y_1, grid_world)
    elif ((y-1 == y_1) and (x == x_1)):#向上走一步
        # Are (x, y) next to (at right of) the (x_1, y_1)?
        return NextStepProb(x_1, y_1, grid_world)
    elif ((y+1 == y_1) and (x == x_1)):#向下走一步
        # Are (x, y) next to (at left of) the (x_1, y_1)?
        return NextStepProb(x_1, y_1, grid_world)    
    else:# cell 1 == cell 2，或者 很远不可能一步走到的情况
        return 0    

def getDistance(towerlocation, grid_world):
    _prob_m = []
    _dm = [] #Distance matrix
    _o = []  #Observations
    _max_distance = 0
    _act_dist = 0 #actual distance between cell to tower，计算出的实际距离
    _min_dist = 0 #Due to noise, the minimum distance return to robot will be 0.7*d，最小噪声距离
    _max_dist = 0 #Due to noise, the minimum distance return to robot will be 1.3*d，最大噪声距离
    _prob = 0.0
    _decimal_unit = 0.1**DECIMAL_PRECISION#小数点到 0.1
    _tx, _ty = 0, 0 #the coordinate of tower
    GRID_WORLD_ROW, GRID_WORLD_COLUMN = np.array(grid_world).shape
    #所有cell里，两个cell的最大实际距离，即对角线距离
    _max_distance = math.sqrt((GRID_WORLD_ROW-1)**2 + (GRID_WORLD_COLUMN-1)**2)

    #从0 到 最大噪声距离，保留一位小数，_o=166个
    for i in range(0, int(_max_distance * NOISY_MAX/_decimal_unit)+1): #+1 ：range 右侧不包括
        _o.extend([round(i*_decimal_unit,DECIMAL_PRECISION)])
    
    for tower in towerlocation:
        _prob_m = []
        for cell in STATES:
            _tx = tower[0]
            _ty = tower[1]
            #每个cell 到 每个塔 的实际距离
            _dist = math.sqrt((VALIDCELL2COORDINATION[cell][0]-_tx)**2+(VALIDCELL2COORDINATION[cell][1]-_ty)**2)
            _min_dist = round(NOISY_MIN * _dist, DECIMAL_PRECISION)
            _max_dist = round(NOISY_MAX * _dist, DECIMAL_PRECISION)
            #每个cell 到 每个塔 的概率，即最大-最小噪声距离
            _prob = 1/((_max_dist - _min_dist + _decimal_unit)/_decimal_unit) # probability of distance between cell to tower for each state 
            _prob_m.append([])
            #如果在 0 到 全部cell里最大的噪声距离中，存在一个值i，这个cell的最小噪声距离<i<最大噪声距离，则该距离概率为_prob
            for i in _o:
                if (_min_dist <= i and i <= _max_dist):
                    _prob_m[cell-1].extend([_prob])
                else:#
                    _prob_m[cell-1].extend([0])  
        #4* _o的个数
        _dm.append(_prob_m)
    return _dm, _o


def getCPT(grid_world):
    _s = [] #states的标号 从1到87
    _tm = [] #transition matrix
    _ip = [] #initial probability
    _total_cells = 0#cell总数
    _valid_cells = 0#有多少个可用的cell
    _x = 0#一共几行
    _y = 0#每行几个cell=一共几列
    _x_1 = 0
    _y_1 = 0
    _valid2coord = {}#可用cell的坐标
    _state = 0 #total _valid_cells+1 states, from X(0) ~ X(_valid_cells).
    
    # Match the state to x, y coordination. 
    for _r in grid_world: #row is x
        for _c in _r: #column is y
            if _c == 1:
                _valid2coord[(_valid_cells+1)] =[_x, _y]
                _s.extend([_valid_cells+1]) #From state1
                _valid_cells += 1 
                _total_cells += 1
                _y += 1
            else:
                _total_cells +=1
                _y += 1
        _y = 0 # 下一个循环是新的一行计数
        _x += 1
        
    # Compute probability for S0 to S1
    # init_p：1/87=0.01149425
    init_p = 1/_valid_cells #Same probability for each valid cell.
    #所有cell的初始化的概率矩阵：1*87
    _ip = np.ones((1, _valid_cells))*init_p
    print('total cells=%d, valid cells=%d, initial probability=%f' % (_total_cells, _valid_cells, init_p))
    
    # Compute probability for S1 for X2. 转换矩阵
    #87*87
    _tm = np.zeros((_valid_cells, _valid_cells))
    # Define transition matrix
   
    for _r in range (_valid_cells): #0...86
        for _c in range (_valid_cells):#0...86
            #从1到87每一个的坐标，t时间的cell 1
            _x_1, _y_1 = _valid2coord[_r+1] 
            if DEBUG > 1: print('x=%d, y=%d'%(_x, _y))
            #从1到87每一个的坐标，t+1的cell 2
            _x, _y = _valid2coord[_c+1] 
            # 计算上一次在cell 1（_x_1, _y_1）的情况下，下一步到cell 2 （_x, _y）的概率 
            _tm[_r][_c] = GetProbability(_x_1, _y_1, _x, _y, grid_world)
            
    return _s, _ip, _tm, _valid2coord


class HMM(object):
    def __init__(self, states, observations, init_prob=None, transition_prob=None, emission_prob=None, algorithm='viterbi'):
        self.states = states
        self.states_len = len(states)
        self.observations = observations
        self.init_prob = init_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob
        self.trace_back_prob = [] #record down highest probability at each time steps
        self.trace_back_states = []
        self.algorithm = algorithm
        if algorithm == 'viterbi':
            self.decode = self.viterbi
        else :
            raise ValueError('Unknown algorithm: %s'%(algorithm))           
        
    def viterbi(self, observation_seq):
        _obs_seq = observation_seq
        _time_steps = 0
        _max_prob = 0
        _states_max_prob = [] # The highest probability of each state at time_step t
        _previous_states = [] # The previous state link to current state
        _states_seq = [] # The out put of state sequences.
        _em_prob = [] #The integrated emission probability from multiple emission probability tables.
        if (np.array(_obs_seq).ndim == 1): #add sequence from one to two dimensions
            _obs_seq = [observation_seq] #不会执行
        else:
            pass #不会执行
        if (np.array(self.emission_prob).ndim ==1):
            self.emission_prob = [self.emission_prob] #不会执行
        else:
            pass #不会执行
        
        _time_steps = len(observation_seq) #11步
        if (_time_steps == 0 or len(self.init_prob[0]) == 0 ): #If input observation sequence is zero length.
            return _states_seq #不会执行
        else:
            pass #不会执行
               
        for step in range(_time_steps): #0...11
            #_em_prob：1*87 （4*87*166的距离矩阵，按列相乘，变成1*87）
            _em_prob = self.eliminate_evidences(_obs_seq, step) #Get the probability of state from given observation sequence of evidences
            if step == 0: #第一步的最大概率，即为初始化的概率，1/87 ：从87个cell里随机选一个
                _states_max_prob = self.init_prob * _em_prob
            
            else:
                #1*87：每走一步，选出每个cell2里所有cell1 概率最大的，以及对应的cell1（_states_max_prob * self.transition_prob，每次都乘上一次最大概率）
                _states_max_prob, _previous_states = self.max_prob_state(_states_max_prob * self.transition_prob) 
                #最大概率*距离
                _states_max_prob = _states_max_prob * _em_prob
                if DEBUG > 0: print('step', step)
                if DEBUG > 0: print('_states_max_prob:', _states_max_prob)
                if DEBUG > 0: print('_em_prob:', _em_prob)
                self.trace_back_states.append(_previous_states)#_previous_states：1*87
        #_states_max_prob：11步结束后的87个cell的最大概率，1*87
        #返回的是最大概率对应的cell
        _states_seq.extend([self.max_prob(_states_max_prob)]) #Record down the highest probability of state in last time step.
        #back track to each time steps for the highest probability of state in each time step.
        #从第11步向0回溯，trace_back_states：11*87
        self.trace_back_states.reverse()
        for _previous_state_list in self.trace_back_states:
            _states_seq.extend([_previous_state_list[_states_seq[-1]]])
        _states_seq.reverse()    
        return _states_seq
    
    def eliminate_evidences(self, observation_seq, step):
        _state_prob = np.ones((1, self.states_len)) #1*87
        _tmp_state_prob = []
        _emission_m = []
        _step = step
        
        #距离矩阵从三维（本来是4*87*166）降至2维，4*87，_tmp_state_prob
        for i, _ev in enumerate(observation_seq[step]): #evidence i at step of time
            _tmp_state_prob.append([])
            for j, _st in enumerate(self.states): # state j
                for k, _obs in enumerate(self.observations): #observation即为_o,166个
                    if(_ev == _obs): #If evidence value is match to the label of observations.
                        _tmp_state_prob[i].extend([self.emission_prob[i][j][k]]) #Get the probability from emission probability matrix.
                        break
                    else:
                        pass

        # _state_prob：4*87的距离矩阵，按列相乘，变成1*87
        for _tmp_prob in _tmp_state_prob: #Consolidate evidence from multiple to one
            _state_prob *= _tmp_prob 

        if DEBUG > 0: print('Print the elimination results for step %d: %s'%(step, _state_prob)) #不会执行
        return _state_prob #1*87
        
    def max_prob_state(self, states_prob):
        # To get maximum probability of state list at time t
        # It will return max_prob list for each state at time t and its previous state at time t-1 
        # For tie breaking, please always prefer the one with a smaller x coordinate, and a smaller y coordinate if the x coordinates are equal.
        _sts_prob = states_prob
        _prob = 0
        _max_prob = 0
        _max_pre_state = 0
        _state_len = self.states_len
        _max_probs_list = []
        _max_pre_states_list = []
        
        for _current_state in range(_state_len): #0...87
            _max_prob = 0
            _previous_state = 0
            for _previous_state in range(_state_len):
                _prob = _sts_prob[_current_state][_previous_state]
                if (_prob > _max_prob):
                    _max_prob = _prob
                    _max_pre_state = _previous_state
                else:
                    pass 
            _max_probs_list.extend([_max_prob])  
            _max_pre_states_list.extend([_max_pre_state]) 
        return _max_probs_list, _max_pre_states_list

    def max_prob(self, states_prob):
        # To get maximum probability of state at time t
        # For tie breaking, please always prefer the one with a smaller x coordinate, and a smaller y coordinate if the x coordinates are equal.
        _sts_prob = states_prob
        _state = 0
        _max_prob = 0
        
        for _st, _prob in enumerate(_sts_prob[0]): #Review row0 first, then each column. row1, column 0 ~ 9...etc.
            if (_prob > _max_prob):
                _max_prob = _prob
                _state = _st
            else:
                pass    
        return _state
    
def state2coordinate(sequence_list, labels_dictionary):
    _seq_list = sequence_list
    _label_dic = labels_dictionary
    _label_seq = []
    
    for _seq in _seq_list:
        _label_seq.extend([_label_dic[_seq+1]]) #starting state from 1 rather than 0.
    
    return _label_seq
'''
    Main program for the HMM class execution.
'''
           
if __name__ == '__main__':
    '''
        Main program.
            Read the observation data from hmm-data.txt.
            Construct HMM with states, observation...etc. data.
            Calculate the sequences of the state.
    '''      
    GRID_WORLD, TOWER_LOCATION, NOISY_DISTANCES = load_parameters(INPUT_FILE)
    if DEBUG > 0: Print_List(GRID_WORLD)
    if DEBUG > 0: print ('TOWER_LOCATION', TOWER_LOCATION)
#     NOISY_DISTANCES.reverse()
    print ('NOISY_DISTANCES', NOISY_DISTANCES)

    STATES, INITIAL_PROB, TRANSITION_MATRIX, VALIDCELL2COORDINATION = getCPT(GRID_WORLD)
    print('STATES,', STATES)
    if DEBUG > 0: print('INITIAL_PROB,', INITIAL_PROB)
    if DEBUG > 0: print('TRANSITION_MATRIX,')
    if DEBUG > 0: Print_List(TRANSITION_MATRIX)

    DISTANCE_MATRIX, OBSERVATIONS = getDistance(TOWER_LOCATION, GRID_WORLD)
    print('OBSERVATIONS=', OBSERVATIONS)
    if DEBUG > 0: print ('DISTANCE_MATRIX for each tower')
    if DEBUG > 0: Print_List(DISTANCE_MATRIX)
    
   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    hmm = HMM(STATES, OBSERVATIONS, INITIAL_PROB, TRANSITION_MATRIX, DISTANCE_MATRIX, 'viterbi')
    state_sequence = hmm.decode(NOISY_DISTANCES)#序列结果
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    print ('State sequence=', [x+1 for x in state_sequence]) #Because we define the first state counts from 1
    print ('Coordination sequence=', state2coordinate(state_sequence, VALIDCELL2COORDINATION))
 
        

NOISY_DISTANCES [[6.3, 5.9, 5.5, 6.7], [5.6, 7.2, 4.4, 6.8], [7.6, 9.4, 4.3, 5.4], [9.5, 10.0, 3.7, 6.6], [6.0, 10.7, 2.8, 5.8], [9.3, 10.2, 2.6, 5.4], [8.0, 13.1, 1.9, 9.4], [6.4, 8.2, 3.9, 8.8], [5.0, 10.3, 3.6, 7.2], [3.8, 9.8, 4.4, 8.8], [3.3, 7.6, 4.3, 8.5]]
total cells=100, valid cells=87, initial probability=0.011494
STATES, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
OBSERVATIONS= [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6.0, 6.1, 6